In [10]:
!pip install -U -q   "faiss-cpu" "tiktoken" "sentence-transformers" "anthropic" "python-dotenv" "pypdf" "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "PyPDF2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!mkdir -p data
!wget https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico -O "data/ppc.pdf"

--2024-04-07 23:17:03--  https://www.uffs.edu.br/atos-normativos/ppc/ccccch/2017-0002/@@download/documento_historico
Resolving www.uffs.edu.br (www.uffs.edu.br)... 200.135.49.107
Connecting to www.uffs.edu.br (www.uffs.edu.br)|200.135.49.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1493834 (1.4M) [application/pdf]
Saving to: ‘data/ppc.pdf’

data/ppc.pdf        100%[===================>]   1.42M  1.44MB/s    in 1.0s    

2024-04-07 23:17:05 (1.44 MB/s) - ‘data/ppc.pdf’ saved [1493834/1493834]



In [5]:
from google.colab import userdata

def write_to_file(filename, text):
    try:
        with open(filename, 'a+') as file:
            file.write('\n'+text)
    except Exception as e:
        print(f"Error writing to file: {e}")

write_to_file('prompts.txt', 'teste2')


class Settings:
  ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
  HF_TOKEN = userdata.get('HF_TOKEN')


settings = Settings()


In [79]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore


def extract_text_from_pdf():
  loader = PyPDFLoader("data/ppc.pdf")
  pages = loader.load_and_split()
  return pages


def split_into_chunks(pages):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000, # the character length of the chunk
    chunk_overlap = 200, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

  return text_splitter.transform_documents(pages)

def init_embeddings_model(embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'):

 return HuggingFaceEmbeddings(model_name=embed_model_id), embed_model_id

def create_vector_store(core_embeddings_model, documents, emdbed_model_id):
  store = LocalFileStore("./cache/")

  embedder = CacheBackedEmbeddings.from_bytes_store(
      core_embeddings_model, store, namespace=embed_model_id
  )

  vector_store = FAISS.from_documents(documents, embedder)

  return vector_store

def save_vector_store(vector_store):
  vector_store.save_local('ppc-index')

In [80]:
pages = extract_text_from_pdf()
documents = split_into_chunks(pages)

In [81]:
core_embeddings_model, embed_model_id = init_embeddings_model('sentence-transformers/distiluse-base-multilingual-cased-v1')

vector_store = create_vector_store(core_embeddings_model, documents, embed_model_id)


In [82]:
query = "Qual o limite de horas para validar atividades complementares de estágios nao obrigatórios?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

for page in docs:
  print(page.page_content)

MINISTÉRIO DA EDUCAÇÃO
UNIVERSIDADE FEDERAL DA FRONTEIRA SUL – UFFS
PRÓ-REITORIA DE GRADUAÇÃO
DIRETORIA DE PEDAGÓGICA
IIAtividades Curriculares de Extensão (Limite horas: 200)
CategoriaDescrição da AtividadeLimite de 
HorasHoras por 
EventoDocumento comprobatório
2.1Experiências de trabalho 150⅙ do total de 
horas de parti-
cipaçãoRequerimento aprovado e
relatório aprovado pelo coorde -
nador de estágios do curso.
2.2Estágios não obrigatórios 150⅙ do total de 
horas de parti-
cipaçãoRelatório aprovado pelo coor -
denador de estágios do curso.
2.3Participação na Empresa 
Júnior150½ do total de 
horas de parti-
cipaçãoRelatório aprovado pelo coor -
denador da Empresa Júnior.
2.4Participação em concursos 
relacionados à temática de 
empreendedorismo (ex. Si -
napse inovação, startup 
weekend, hackathons)120total de horas 
de participaçãoDeclaração do responsável pelo
projeto, ou certificado.
2.5Participação como compe -
tidor, em maratonas de pro -
gramação12020 horas por 
eventoTabela de

# Testando com Anthropic Claude

In [77]:
import anthropic

def build_prompt(query, context):
  return f"""Human:\n\n
  Voce é um assistente academico especializado em tirar dúvidas de alunos de ciencia da computação da UFFS.
  Responda com base no contexto:
  {context}
  Responda apenas se encontrar resposta no contexto.
  Pergunta: {query}
  A resposta deve seguir o padrão:
  - Pergunta
  R: Resposta (trecho de referencia para resposta)

  Assistant:
  """

def get_claude_response(prompt):
  client = anthropic.Anthropic(
      api_key = settings.ANTHROPIC_API_KEY
    )
  message = client.messages.create(
      model="claude-instant-1.2",
      max_tokens=200,
      temperature=0.1,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "text",
                      "text": prompt
                  }
              ]
          }
      ]
  )
  return message.content[0].text

def get_answer(query):
  embedding_vector = core_embeddings_model.embed_query(query)
  docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

  context = ''
  for page in docs:
    context += page.page_content



  prompt = build_prompt(context, query)
  write_to_file('prompts.txt', prompt)
  response = get_claude_response(prompt)

  return response



In [83]:
queries = [
    "Qual a ementa de algebra linear?",
    "Qual o limite de horas de atividades curriculares de Pesquisa?",
    "Quantas horas de ACC devo ter para me formar?",
    "Preciso das referencias para a disciplina Engenharia de Software II",
    "Qual o limite de horas para validar atividades complementares de estágios nao obrigatórios?",
    "Qual a ementa de Grafos?"
]

for query in queries:
  print()
  response = get_answer(query)
  print(response)


Pergunta: Qual a ementa de algebra linear?

R: EMENTA
Matrizes. Determinantes. Sistemas de equações lineares. Espaços vetoriais. Transformações
lineares. Espaços com produto interno. Autovalores e autovetores. Diagonalização.

Pergunta: Qual o limite de horas de atividades curriculares de Pesquisa?

R: De acordo com o documento, o limite de horas para as Atividades Curriculares Complementares em Pesquisa e Ensino (Grupo I) é de 300 horas (ou, equivalentemente, 360 horas-aula) e estas atividades englobam atividades de pesquisa, monitorias e grupos de estudos realizadas pelos estudantes.

Pergunta: Quantas horas de ACC devo ter para me formar?

R: De acordo com o Projeto Pedagógico do Curso de Graduação em Ciência da Computação da UFFS, as Atividades Curriculares Complementares (ACCs) constituem ações que visam a complementação do processo ensino-aprendizagem, sendo desenvolvidas com carga horária de 300 horas (ou, equivalentemente, 360 horas-aula) e distribuídas ao longo do período de 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

documents = text_splitter.transform_documents(pages)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# import torch
# import transformers
# from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
# from peft import PeftModel, PeftConfig

# model_id = 'recogna-nlp/bode-7b-alpaca-pt-br'

# config = PeftConfig.from_pretrained(model_id)


# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, trust_remote_code=True, return_dict=True, load_in_8bit=True, device_map='auto')

# model.eval()



import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.1,
    max_new_tokens=512
)

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
query = "Quem é o coordenador do curso?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 5)

context = ''
for page in docs:
  context += page.page_content





prompt = f"""
Voce é um assistente da coordenação academica do curso de ciencia da computação da UFFS.
Com base no contexto: <contexto> {context}. </contexto>. Responda, em portugues, de acordo com o contexto, a pergunta: {query}.
A saída deve ser no formato:
  - Pergunta
  - R: resposta

"""

print(prompt)

response = llm(prompt[0])

print(response)


In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

In [ ]:
qa_with_sources_chain({"query" : "Qual a ementa de geometria analíica?"})